In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
#geemap.update_package()

In [3]:
##converting javascript code to python (also in map interface):
#js_snippet = """
#"""
#geemap.js_snippet_to_py(js_snippet, add_new_cell = True, import_ee = False, import_geemap = False, show_map = False)

In [4]:
#list of plots in Virginia LTER dataset (not including broad creek)
ee.Initialize()

plot_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-75.372740, 37.903114]), {'site_id': 'Assateague'}), #Assateague
  ee.Feature(ee.Geometry.Point([-75.671577, 37.625273]), {'site_id': 'Bellvue'}), #Bellvue
  ee.Feature(ee.Geometry.Point([-75.876094, 37.395746]), {'site_id': 'Box_Tree'}), #Box_Tree
  ee.Feature(ee.Geometry.Point([-75.652248, 37.709249]), {'site_id': 'Channel_Point'}), #Channel_Point
  ee.Feature(ee.Geometry.Point([-75.942331, 37.175055]), {'site_id': 'Cushmans'}), #Cushmans
  ee.Feature(ee.Geometry.Point([-75.940586, 37.167409]), {'site_id': 'Gator'}), #Gator
  ee.Feature(ee.Geometry.Point([-75.814954, 37.485033]), {'site_id': 'Greens_Creek'}), #Greens_Creek
  ee.Feature(ee.Geometry.Point([-75.672871, 37.451132]), {'site_id': 'Hog'}), #Hog
  ee.Feature(ee.Geometry.Point([-75.900996, 37.346190]), {'site_id': 'Indiantown'}), #Indiantown
  ee.Feature(ee.Geometry.Point([-75.558264, 37.784154]), {'site_id': 'Kegotank'}), #Kegotank
  ee.Feature(ee.Geometry.Point([-75.861733, 37.416783]), {'site_id': 'Mill_Creek'}), #Mill_Creek
  ee.Feature(ee.Geometry.Point([-75.822928, 37.468043]), {'site_id': 'N_Brownsville'}), #N_Brownsville
  ee.Feature(ee.Geometry.Point([-75.929133, 37.287549]), {'site_id': 'Oyster'}), #Oyster
  ee.Feature(ee.Geometry.Point([-75.940690, 37.181277]), {'site_id': 'Steelmans'}), #Steelmans
  ee.Feature(ee.Geometry.Point([-75.447000, 37.880000]), {'site_id': 'Wallops'}), #Wallops
  ee.Feature(ee.Geometry.Point([-75.819360, 37.481998]), {'site_id': 'Woodland_Farm'}) #Woodland_Farm
])


ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
])
##for GA, lats and longs may differ slightly between years (compare average lat/long to recorded lat/longs in each yr)

In [5]:
##landsat 5 surface reflectance images - VA LTER

roi = ee.Geometry.Point(-75.876094, 37.395746)

filteredCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2000,'year')) \
    .filter(ee.Filter.calendarRange(7, 9,'month')) \
    .filterBounds(plot_pts) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS5.size().getInfo())
image0 = filteredCollectionLS5.first()
#print(image0.getInfo())

3


In [6]:
##landsat 8 surface reflectance images - VA LTER
filteredCollectionLS8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2014, 2015,'year')) \
    .filter(ee.Filter.calendarRange(7, 9,'month')) \
    .filterBounds(plot_pts) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS8.size().getInfo())

8


In [7]:
#landsat 7 surface reflectance images - VA LTER - might not be able to use due to striping
filteredCollectionLS7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(1999, 2017,'year')) \
    .filter(ee.Filter.calendarRange(8, 9,'month')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS7.size().getInfo())

61


In [8]:
##get list of image ids in filtered collection for L5, L7, and L8
# filteredCollectionLS5.aggregate_array('system:id').getInfo()
# filteredCollectionLS8.aggregate_array('system:id').getInfo()
# filteredCollectionLS7.aggregate_array('system:id').getInfo()

In [9]:
##add image number for each of the images in the filtered collection - closer to 8/15 is best
##Check sensing time of image - should be in GMT/UTC?
##Tide requirements: above xxx m

##landsat 5
image99 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_19990729')

# image00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20000901') #Flooding
image00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20000816') ##Slightly better, perhaps

image01 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20010803') 

# image02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20020822') # Flooding, clouds
image02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20020705') #Not optimal, but slightly better

image03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20030825')
image04 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20040811')
image05 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20050814') #Some clouds
image06 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20060817') 
image07 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20070905')
image08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20080822')
image09 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20090809') #Flooded, but other scenes are higher tides
image10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20100828') #Cloudy, but best option
image11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20110815')

##landsat 7
image12 = ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_014034_20120926') #l7 striping is fixed in study area! BUT NEED UNFILLED DATA

##landsat 8
image13 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20130905') 

# image14 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20140807') 
image14 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20140722') ##A couple weeks before sampling date, but much better clouds

# image15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20150826') 
image15 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20150725') ##A couple weeks before sampling date, but much better clouds

image16 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20160812') 
image17 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20170831') 
image18 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_014034_20180919')

##Make image collection for each year, then cloud mask? Rather than cloud masking individual images?

l8_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B5', 'B4', 'B3'], 
             'gamma': 1.5} ##be very careful with analysis of l8 vs l5 data - different band numbers!

#Map.addLayer(image15, l8_param, "l82015")
#Map.addLayer(image12, vis_param, "l72012")

props = geemap.image_props(image00)
props.getInfo()

{'CLOUD_COVER': 19,
 'CLOUD_COVER_LAND': 18,
 'EARTH_SUN_DISTANCE': 1.01247,
 'ESPA_VERSION': '2_19_0c',
 'GEOMETRIC_RMSE_MODEL': 4.379,
 'GEOMETRIC_RMSE_MODEL_X': 2.708,
 'GEOMETRIC_RMSE_MODEL_Y': 3.441,
 'IMAGE_DATE': '2000-08-16',
 'IMAGE_QUALITY': 9,
 'LANDSAT_ID': 'LT05_L1TP_014034_20000816_20160918_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1474222091000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.5.0',
 'SATELLITE': 'LANDSAT_5',
 'SENSING_TIME': '2000-08-16T15:18:19.4160250Z',
 'SOLAR_AZIMUTH_ANGLE': 127.416634,
 'SOLAR_ZENITH_ANGLE': 34.107227,
 'SR_APP_VERSION': 'LEDAPS_3.2.0',
 'WRS_PATH': 14,
 'WRS_ROW': 34,
 'system:asset_size': '283.768603 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'sr_atmos_opacity',
  'sr_cloud_qa',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20000816',
 'system:index': 'LT05_014034_20000816',
 'system:time_end': '2000-08-16 15:18:19',
 'system:time_start': '2000-08

In [10]:
##Mapping
##Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(image02, vis_param, "2016")

Map.centerObject(plot_pts, 10)
#Map.set_plot_options(add_marker_cluster = True) ##not sure what this does


##add feature collection points for each plot
Map.addLayer(plot_pts)

Map

Map(center=[37.485005614872954, -75.76448856851104], controls=(WidgetControl(options=['position', 'transparent…

In [17]:
##Coordinates for each sampling transect

allplots_fc= pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Virginia/Coordinates and biomass/va_allpoints.csv')
allplots_fc = allplots_fc[allplots_fc['liveMass'].notna()]
allplots_fc = allplots_fc[allplots_fc['Latitude'].notna()]
allplots_fc = allplots_fc.drop(columns = ['deadMass', 'collectDate', 'totalMass', 'Ell', 'MSL'])

display(allplots_fc)

fc_all = geemap.pandas_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_all, {}, "fc_all") 

,Year,marshName,siteName,locationID,monitoringPurpose,Transect,Replicate,speciesName,liveMass,latitude,longitude,Code,Latitude,Longitude
16,1999,Box Tree,Box_Tree,1,EOYB,C,b,Spartina alterniflora,0.00,37.395746,-75.876094,BTCC,37.396051,-75.876056
17,1999,Box Tree,Box_Tree,3,EOYB,C,b,Spartina alterniflora,272.48,37.395746,-75.876094,BTC2,37.396450,-75.877045
18,1999,Box Tree,Box_Tree,1,EOYB,D,b,Spartina alterniflora,0.00,37.395746,-75.876094,BTDC,37.396337,-75.875969
19,1999,Box Tree,Box_Tree,1,EOYB,A,b,Spartina alterniflora,0.00,37.395746,-75.876094,BTAC,37.395461,-75.876249
20,1999,Box Tree,Box_Tree,4,EOYB,A,b,Spartina alterniflora,0.00,37.395746,-75.876094,BTA3,37.396366,-75.878189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440,2017,Oyster,Oyster,1,EOYB,B,b,Spartina alterniflora,480.00,37.287549,-75.929133,OSBC,37.287549,-75.929133
2441,2017,Oyster,Oyster,1,EOYB,B,a,Spartina alterniflora,402.88,37.287549,-75.929133,OSBC,37.287549,-75.929133
2448,2017,Gator Tract,Gator,3,EOYB,C,a,Spartina alterniflora,42.72,37.167409,-75.940586,GTC2,37.167768,-75.944280
2449,2018,Box Tree,Box_Tree,1,EOYB,C,a,Spartina alterniflora,255.04,37.395746,-75.876094,BTCC,37.396051,-75.876056


In [12]:
## This example demonstrates the use of the pixel QA band to mask
## clouds in surface reflectance (SR) data.  It is suitable
## for use with any of the Landsat SR datasets.

##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  ##Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

##Map the function over image collection
collection = filteredCollectionLS5 \
    .map(maskL5sr)

composite = collection.median() #what exactly is this doing? median pixel value?

##Display the results:

#Map.addLayer(composite, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "cloudfree composite")

In [13]:
##Water masking

##Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')

##Select the land/water mask.
datamask = hansenImage.select('datamask')

##Create a binary mask.
watermask = datamask.eq(1)

##Works, but not as well as in GA sites - masks out a little too much, I think - entire islands missing

In [14]:
masked00 = ee.Image(maskL5sr(image00))
#Map.addLayer(composite) #using cloud mask on cloudfree composite with default vis param as gray background
#Map.addLayer(masked00, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, "2000 masked") #isn't perfect but good
#Map.addLayer(image00, vis_param, "2000 unmasked")
#Map.addLayer(collection.first(), {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.3}, "masked???")
#Map.addLayer(plot_pts)

In [15]:
##cloud- and cloud shadow-masked images - VA LTER :D
masked99 = ee.Image(maskL5sr(image99)).updateMask(watermask)
masked00 = ee.Image(maskL5sr(image00)).updateMask(watermask)
masked01 = ee.Image(maskL5sr(image01)).updateMask(watermask)
masked02 = ee.Image(maskL5sr(image02)).updateMask(watermask)
masked03 = ee.Image(maskL5sr(image03)).updateMask(watermask)
masked04 = ee.Image(maskL5sr(image04)).updateMask(watermask)
masked05 = ee.Image(maskL5sr(image05)).updateMask(watermask)
masked06 = ee.Image(maskL5sr(image06)).updateMask(watermask)
masked07 = ee.Image(maskL5sr(image07)).updateMask(watermask)
masked08 = ee.Image(maskL5sr(image08)).updateMask(watermask)
masked09 = ee.Image(maskL5sr(image09)).updateMask(watermask)
masked10 = ee.Image(maskL5sr(image10)).updateMask(watermask)
masked11 = ee.Image(maskL5sr(image11)).updateMask(watermask)
masked12 = ee.Image(maskL5sr(image12)).updateMask(watermask) ##may need to remove - landsat 7; or just look at stripes
masked13 = ee.Image(maskL5sr(image13)).updateMask(watermask)
masked14 = ee.Image(maskL5sr(image14)).updateMask(watermask)
masked15 = ee.Image(maskL5sr(image15)).updateMask(watermask)
masked16 = ee.Image(maskL5sr(image16)).updateMask(watermask)
masked17 = ee.Image(maskL5sr(image17)).updateMask(watermask)
masked18 = ee.Image(maskL5sr(image18)).updateMask(watermask)

##Check masked images:
Map.addLayer(masked16, {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, "masked16")
Map.addLayer(plot_pts)

##Creating an image collection:
va_finalcollection = ee.ImageCollection.fromImages([masked99, masked00, masked01, masked02, masked03, masked04, \
                                                    masked05, masked06, masked07, masked08, masked09, \
                                                    masked10, masked11, masked12, masked13, masked14, \
                                                    masked15, masked16, masked17])

landsat5_final = ee.ImageCollection.fromImages([masked99, masked00, masked01, masked02, masked03, masked04, \
                                                masked05, masked06, masked07, masked08, masked09, \
                                                masked10, masked11])

landsat7_final = ee.ImageCollection.fromImages([masked12])

landsat8_final = ee.ImageCollection.fromImages([masked13, masked14, masked15, masked16, masked17, \
                                                masked18])

In [16]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

##Flooding potential = 1 - 1/e^(-1.6+20*ndwi+68.6*pheno) - logistic regression
##Identical formula: 1/e^-(-1.6+20*ndwi+68.6*pheno)

def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

##Notes: This is setup for Landsat 8 - apply to l5? Also, had trouble with np.exp() in equation, so used approx value for e
##np.exp() only takes one argument

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))

mergedFLATS = landsat8_final.map(addFLATS)

FLAT16 = addFLATS(masked16)

# Map.addLayer(mergedFLATS.first(), {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, 'Flats')
Map.addLayer(FLAT16, {'bands': ['flats'], 'min': 0, 'max': 1}, 'Flats 2019')

FLAT18 = addFLATS(masked18)
FLAT17 = addFLATS(masked17)
FLAT16 = addFLATS(masked16)
FLAT15 = addFLATS(masked15)
FLAT14 = addFLATS(masked14)
FLAT13 = addFLATS(masked13)
FLAT12 = addFLATS(masked12)
FLAT11 = addFLATSL5(masked11)
FLAT10 = addFLATSL5(masked10)
FLAT09 = addFLATSL5(masked09)
FLAT08 = addFLATSL5(masked08)
FLAT07 = addFLATSL5(masked07)
FLAT06 = addFLATSL5(masked06)
FLAT05 = addFLATSL5(masked05)
FLAT04 = addFLATSL5(masked04)
FLAT03 = addFLATSL5(masked03)
FLAT02 = addFLATSL5(masked02)
FLAT01 = addFLATSL5(masked01)
FLAT00 = addFLATSL5(masked00)
FLAT99 = addFLATSL5(masked99)

Map

Map(bottom=101891.0, center=[37.485005614872954, -75.76448856851104], controls=(WidgetControl(options=['positi…

In [17]:
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m. Also I don't know what the data looks like
dem3 = ee.Image('USGS/SRTMGL1_003') ##1 arc second, or 30 m, and large coverage. Elevation is in meters (from GDEM datum?)

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem3, dem_params, 'SRTM DEM', True, 1)
Map.addLayer(dem, dem_params, '10m DEM', True, 1)

##Exporting pixel values for DEM, all plots!
in_fc = fc_all
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'VA_DEM_vals_10m.csv')
# geemap.extract_values_to_points(in_fc, dem, out_csv)

In [18]:
##Exporting pixel values - any way to do zonal averages without manually selecting - VA LTER
coords_year = pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Virginia/Coords_year/coords_2015.csv')

coords_year = coords_year.drop(columns=['collectDate','Ell','MSL', 'deadMass', 'totalMass'])
coords_year = coords_year[coords_year['Latitude'].notna()] ##Removing plots without specific lat/longs
coords_year = coords_year[coords_year['liveMass'].notna()] ##Removing plots without biomass

fc_coords = geemap.pandas_to_ee(coords_year, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_coords, {}, "fc_coords")

##export data
in_fc = fc_coords

out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'flats15.csv')

# geemap.extract_values_to_points(in_fc, FLAT15, out_csv) ##1999-2018

In [ ]:
##Exporting pixel values - zonal statistics: Average 3x3 grid for Landsat \
##(radius = 30 or 90 m - is the 'scale' option in zonal_statistics radius???)

out_dir = os.path.expanduser('~/Downloads')
out_va_stats = os.path.join(out_dir, 'va_stats.csv')  

#geemap.zonal_statistics(in_value_raster = va_finalcollection, in_zone_vector = plot_pts, \
                # out_file_path = out_va_stats, statistics_type='MEAN', \
                # scale = 90)
        
        ##This ^^ is the right idea, but need to clean up bands selected, and still need to figure out a water mask
        ##Also: why the reflectance values so different than VA dataset: Has to do with masking params

In [57]:
##summer median composite pixel values - error when exporting values!!!
LS5comp2000 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2000,'year')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(LS5comp2000.size().getInfo())

collection2000 = LS5comp2000.map(maskL5sr)
composite2000 = collection2000.median()
#Map.addLayer(composite2000, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.3}, "cloudfree composite 2000")

16


In [51]:
#LS5comp2000.aggregate_array('system:id').getInfo()

In [29]:
print("hi")

hi
